In this notebook, we generate a Feed Forward Neural Network.

In [2]:
import os # Useful for running command line within python
import pandas as pd ## Useful for data manipulation

import torch ## Pytorch is the deep learning library that we will be using
import torch.nn as nn # Neural network module
import torch.nn.functional as F ## Functional module
import torchmetrics ## Torchmetrics is a library that contains metrics for evaluating models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets.folder import default_loader

import pytorch_lightning as pl ## Pytorch lightning is a wrapper for pytorch that makes it easier to train models
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import Callback, ModelCheckpoint
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from torchmetrics.regression import MeanSquaredError

from torch.nn import ReLU, Sequential

I need to load the trainset, valset, and testset.

In [5]:
data=pd.read_csv("./data/cleaned_data/86077_aws_plumber_data.csv")

Define dataloaders

In [13]:
(data["Precipitation_since_9am_local_time_in_mm"]+data["Air_Temperature_in_degrees_C"])/data["Wind_speed_in_kmh"]

0        1.468085
1        1.468085
2        1.855263
3        1.553192
4        1.947368
           ...   
87595    1.119565
87596    1.584615
87597    1.830357
87598    1.538462
87599    1.500000
Length: 87600, dtype: float64

In [16]:
np.sqrt(data["Wind_speed_in_kmh"])

0        3.065942
1        3.065942
2        2.756810
3        3.065942
4        2.756810
           ...   
87595    4.289522
87596    3.605551
87597    3.346640
87598    3.605551
87599    3.605551
Name: Wind_speed_in_kmh, Length: 87600, dtype: float64

In [15]:
import numpy as np

In [11]:
13.8/9.4

1.4680851063829787

In [12]:
14.1/7.6

1.855263157894737

In [3]:
## Feed Forward Neural Network

class feed_forward(pl.LightningModule):
    def __init__(self, n, learning_rate=1e-2, traindataloader=None, valdataloader=None, testdataloader=None):
        ## n is the number of inputs
        super().__init__()
        
        # define learning rate
        self.learning_rate = learning_rate ## Just the numerical value of the learning rate

        # define loss function (Cross entropy loss)
        self.loss_fun = nn.CrossEntropyLoss() ## The function to calculate the loss

        # define neural layers
        ## The architectures follows the NN defined in the Urban Heat Island paper about Athens 
        self.linear1 =  nn.Linear(n, 30) ## 
        self.linear2 =  nn.Linear(30, 30) ##
        self.linear3= nn.Linear(30,1) ## This layer makes a prediction.
        
        # Create ReLU Activation Layer
        self.relu=nn.ReLU() ####

        # We calculate the mean square error as a perfomance metric
        
        self.train_mse = MeanSquaredError()
        self.val_mse = MeanSquaredError()
        self.test_mse = MeanSquaredError()

        # Define dataloaders
        self.traindataloader = traindataloader ### I am just passing the corresponding dataloader
        self.valdataloader = valdataloader   ### I am just passing the corresponding dataloader
        self.testdataloader = testdataloader ### I am just passing the corresponding dataloader

    def forward(self, x):
        
        out1=self.relu(self.linear1(x))
        out2=self.relu(self.linear2(out1))
        out=self.linear3(out2)

        return out # return the output

    def training_step(self, batch, batch_idx):

        # Make predictions
        x,y=batch ## The batch should come from the CovidDataset class
        # Do the prediction
        
        y_hat=self(x) ## This predicts the results from x

        # Calculate the loss
        # Apply the loss function here
        loss=self.loss_fun(y_hat,y) ## Calculate the loss with the function declared in this class
        ## Update the MSE
        self.train_mse.update(y_hat, y)
        
        
        # Record accuracy and loss
        # Calling self.log will surface up scalars for you in TensorBoard
        # You may add more logs as you think necessary
         
        ### In the next two lines the history of the loss and accuracy is logged
        self.log("train_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("train_mse", self.train_mse, prog_bar=True, on_step=False, on_epoch=True)

        # Return the loss
        return loss ## 

    def validation_step(self, batch, batch_idx):
        
        ### I am going to reuse the code for the training
        x,y=batch 
        y_hat=self(x) ## This predicts the results from x

        # Compute loss for each batch
        loss=self.loss_fun(y_hat,y) ## Calculate the loss 
        ## Update the MSE
        self.val_mse.update(y_hat, y)


        # Record accuracy and loss
        # Log anything you think necessary
        ### Save the logs for the loss and for the accuracy
        self.log("val_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("val_mse", self.val_mse, prog_bar=True, on_step=False, on_epoch=True)

    def test_step(self, batch, batch_idx):
        
        x,y=batch 
        y_hat=self(x) ## This predicts the results from x

        # Compute loss for each batch
        loss=self.loss_fun(y_hat,y) ## Calculate the loss 

        self.test_mse.update(y_hat, y) 

        # Record accuracy and loss
        # Log anything you think necessary
        self.log("test_loss", loss, prog_bar=True, on_step=False, on_epoch=True)
        self.log("test_mse", self.test_mse, prog_bar=True, on_step=False, on_epoch=True)

    def predict_step(self, batch, batch_idx):
        
        x,y=batch
        # Do prediction
        y_hat=self(x)

        return y_hat,y,x # Return prediction, actual value and inputs  

    def configure_optimizers(self):
        # define optimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    ####################
    # DATA RELATED HOOKS
    ####################

    def train_dataloader(self):
        # return the train dataloader
        return  self.traindataloader ## Just return the traindataloader

    def val_dataloader(self):
        # return the validation dataloader
        return self.valdataloader  ### Just return the valdataloader

    def test_dataloader(self):
        # return the test dataloader
        return self.testdataloader  ## Just return the testdataloader